In [15]:
import numpy as np
import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [16]:
train_data, validation_data, test_data = tfds.load(
name="imdb_reviews",
split=('train[:60%]', 'train[60%:]', 'test'),
as_supervised=True) # tuple형태로 데이터를 return받는다

In [17]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [21]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [32]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
# TF Hub에서 pre-train된 모델을 가져오기
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                          dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [33]:
model = tf.keras.Sequential() # 모델을 순차적으로 쌓아 올리기
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_6 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])

# BinaryCrossentropy : 확률값 output에는 Crossentropy가 효과적
# from_logits=True 

In [36]:
history = model.fit(train_data.shuffle(10000).batch(512),
                   epochs=50,
                   validation_data=validation_data.batch(512),
                   verbose=1)

# 매 에폭마다 데이터를 섞어줌
# verbose = 0 : 출력x
# verbose = 1 : 진행도 막대 보여줌
# verbose = 2 : mini-batch마다 loss출력

Epoch 1/50
30/30 [==============================] - 1s 50ms/step - loss: 0.6967 - accuracy: 0.5146 - val_loss: 0.6816 - val_accuracy: 0.5252
Epoch 2/50
30/30 [==============================] - 1s 48ms/step - loss: 0.6775 - accuracy: 0.5548 - val_loss: 0.6659 - val_accuracy: 0.6071
Epoch 3/50
30/30 [==============================] - 1s 42ms/step - loss: 0.6595 - accuracy: 0.6311 - val_loss: 0.6499 - val_accuracy: 0.6512
Epoch 4/50
30/30 [==============================] - 1s 41ms/step - loss: 0.6433 - accuracy: 0.6870 - val_loss: 0.6372 - val_accuracy: 0.7095
Epoch 5/50
30/30 [==============================] - 1s 41ms/step - loss: 0.6303 - accuracy: 0.7280 - val_loss: 0.6265 - val_accuracy: 0.7381
Epoch 6/50
30/30 [==============================] - 1s 42ms/step - loss: 0.6184 - accuracy: 0.7603 - val_loss: 0.6170 - val_accuracy: 0.7567
Epoch 7/50
30/30 [==============================] - 1s 41ms/step - loss: 0.6078 - accuracy: 0.7905 - val_loss: 0.6094 - val_accuracy: 0.7673
Epoch 8/50
30

In [37]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s : %.3f" % (name, value))

49/49 - 1s - loss: 0.5721 - accuracy: 0.8579
loss : 0.572
accuracy : 0.858
